In [ ]:
import pandas as pd
import json

df = pd.read_csv("nikkei.csv")

data = {
    "start": df["date"].iloc[0],
    "target": df["close"].tolist()
}

with open("train.json", "w") as f:
    f.write(json.dumps(data))

import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

bucket = session.default_bucket()
prefix = "nikkei-deepar"

s3_train_path = session.upload_data(
    path="train.json",
    bucket=bucket,
    key_prefix=f"{prefix}/train"
)

print(s3_train_path)

from sagemaker.estimator import Estimator
import sagemaker.image_uris

region = session.boto_region_name

image_uri = sagemaker.image_uris.retrieve(
    framework="forecasting-deepar",
    region=region
)

estimator = Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",  # 最小構成
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=session
)

estimator.set_hyperparameters(
    time_freq="D",            # 日次データ
    prediction_length=5,      # 7日先予測
    context_length=20,
    epochs=20,
    mini_batch_size=32,
    learning_rate=1e-3
)

estimator.fit({"train": s3_train_path})